### tensorflowのkerasバグ対応
https://github.com/tensorflow/tensorflow/issues/24520

C:\Users\%USERNAME%\Anaconda3\Lib\site-packages\tensorflow\python\keras\engine\training_utils.py  
line: 237  

Before
```Python
def standardize_single_array(x, expected_shape=None):
  """Expand data of shape (x,) to (x, 1), unless len(expected_shape)==1."""
  if x is None:
    return None

  if (x.shape is not None
      and len(x.shape) == 1
      and (expected_shape is None or len(expected_shape) != 1)):
    if tensor_util.is_tensor(x):
      x = array_ops.expand_dims(x, axis=1)
    else:
      x = np.expand_dims(x, 1)
  return x
```

After  
```Python
def standardize_single_array(x, expected_shape=None):
  """Expand data of shape (x,) to (x, 1), unless len(expected_shape)==1."""
  if x is None:
    return None
  #-------------------------------------
  if tensor_util.is_tensor(x):
    x_shape_ndims = array_ops.rank(x)
    return x
  #-------------------------------------
  if (x.shape is not None
      and len(x.shape) == 1
      and (expected_shape is None or len(expected_shape) != 1)):
    if tensor_util.is_tensor(x):
      x = array_ops.expand_dims(x, axis=1)
    else:
      x = np.expand_dims(x, 1)
  return x
```

### 実行環境
tensorflowのDataset APIを活用する場合、tensorflow内のkerasを利用するとエラーが出る不具合がある。
```
ValueError: Cannot take the length of shape with unknown rank.
```
https://stackoverflow.com/questions/53851793/valueerror-cannot-take-the-length-of-shape-with-unknown-rank

In [1]:
import tensorflow as tf
print(tf.__version__)

import tensorflow.keras as keras
print(keras.__version__)

1.13.1
2.2.4-tf


In [2]:
from tensorflow.keras import backend as K
sess = tf.Session()
K.set_session( sess )

In [3]:
import os
os.environ[ 'TF_CPP_MIN_LOG_LEVEL'] = '2'

### GoogleDriveをマウントする。

In [4]:
import os
if os.name == 'nt':
    print('OS is Windows: PASS mount google drive')
    g_dir_work = '../colab/'
else:
    from google.colab import drive
    drive.mount('/content/drive')
    g_dir_work = '/content/drive/My Drive/colab/'

# check mount point
print('\n<< Display work dir >>')
for file in os.listdir(g_dir_work):
    print( 'file/dir : ', file)

OS is Windows: PASS mount google drive

<< Display work dir >>
file/dir :  .ipynb_checkpoints
file/dir :  data
file/dir :  mnist_cams
file/dir :  mnist_pix2pix
file/dir :  models
file/dir :  mylib
file/dir :  template.ipynb
file/dir :  test.txt
file/dir :  tmp


### local Libraryパスを通す。

In [5]:
import sys
if os.name == 'nt':# windows
    lib_path='../colab/mylib'
else:
    print(os.getcwd())
    lib_path='/content/drive/My Drive/colab/mylib/'
sys.path.append(lib_path)

### tf.data.TextLineDatasetのサンプル
https://deepage.net/tensorflow/2017/07/18/tensorflow-dataset-api.html

In [6]:
class Categories(object):
    __instance = None
    def __new__(cls, *args, **keys):
        if cls.__instance is None:
            cls.__instance = object.__new__(cls)
        return cls.__instance

    def __init__(self):
            self.items = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9' ]
            self.num   = len(self.items)

In [7]:
import datasets
import numpy as np

data = datasets.Dataset_mnist()
data.load()
input_file = data.save_for_textlinedataset(num=30)

import numpy as np
work_dir = '../colab/tmp'
input_file = work_dir + '/input.txt'
N = 100
num_labels = Categories().num

with open(input_file, 'w') as fp:
    for i in range(N):
        data = np.random.random(32).astype(np.float32)
        label = int(np.random.random()* num_labels) # ラベル番号:0-(num_labels-1)
        fp.writelines('{0}/{1:03d}.npy,{2}\n'.format(work_dir, i, label))
        np.save('{0}/{1:03d}.npy'.format(work_dir, i), data)

In [8]:
def to_index(label):
    return Categories().items.index(label)

def parse_csv(line):
    [filename, category] = line.decode('utf-8').split(',')
    return filename, to_index(category)

def read_data(filename, label):
    inputs = np.load(filename.decode('utf-8')).astype(np.float32) / 255.0
    inputs = np.reshape(inputs, (inputs.shape[0], inputs.shape[1], 1))
    return inputs, label

def one_hot(data, label):
    return data, tf.one_hot( label, Categories().num  )

dataset = tf.data.TextLineDataset(input_file)
#dataset = dataset.skip(1)   # 列の読み飛ばし
dataset = dataset.map(lambda x  : tf.py_func(parse_csv, [x],    [tf.string,  tf.int32]))
dataset = dataset.map(lambda x,y: tf.py_func(read_data, [x, y], [tf.float32, tf.int32]))
dataset = dataset.map(one_hot)
dataset = dataset.repeat()
dataset = dataset.shuffle(4)
dataset = dataset.batch(4)

iterator = dataset.make_one_shot_iterator()
next_elem = iterator.get_next()

Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    


In [9]:
# datasetテスト
val = sess.run(next_elem)

print(len(val))
print('\ndata ----------------------------------')
print(val[0].shape)
print(val[0].max())
print(val[0].min())
#print(val[0])

print('\nlabels ----------------------------------')
print(val[1].shape)
print(val[1])

2

data ----------------------------------
(4, 28, 28, 1)
1.0
0.0

labels ----------------------------------
(4, 10)
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]


### 訓練用モデル構築


In [10]:
import models
model = models.Model_mnist_classification()
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
imgs (InputLayer)            (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)    

### モデルの最適化

In [11]:
batch_size = 10
epochs = 10
steps_per_epoch = 3

In [13]:
history = model.fit(dataset.make_one_shot_iterator(), epochs=epochs, steps_per_epoch=steps_per_epoch)

Epoch 1/10
3/3 [==============================] - 10s 3s/step - loss: 2.3300 - acc: 0.0000e+00
Epoch 2/10
3/3 [==============================] - 1s 309ms/step - loss: 2.3071 - acc: 0.0833
Epoch 3/10
3/3 [==============================] - 1s 235ms/step - loss: 2.2950 - acc: 0.2500
Epoch 4/10
3/3 [==============================] - 1s 242ms/step - loss: 2.2942 - acc: 0.1667
Epoch 5/10
3/3 [==============================] - 1s 250ms/step - loss: 2.3106 - acc: 0.0000e+00
Epoch 6/10
3/3 [==============================] - 1s 220ms/step - loss: 2.2585 - acc: 0.1667
Epoch 7/10
3/3 [==============================] - 1s 272ms/step - loss: 2.2958 - acc: 0.0000e+00
Epoch 8/10
3/3 [==============================] - 1s 268ms/step - loss: 2.2629 - acc: 0.0833
Epoch 9/10
3/3 [==============================] - 1s 224ms/step - loss: 2.2491 - acc: 0.2500
Epoch 10/10
3/3 [==============================] - 1s 258ms/step - loss: 2.3191 - acc: 0.1667


In [14]:
history = model.fit(dataset, epochs=epochs, steps_per_epoch=steps_per_epoch)

Epoch 1/10
3/3 [==============================] - 8s 3s/step - loss: 2.2635 - acc: 0.1667 
Epoch 2/10
3/3 [==============================] - 1s 250ms/step - loss: 2.3076 - acc: 0.0833
Epoch 3/10
3/3 [==============================] - 1s 252ms/step - loss: 2.2712 - acc: 0.2500
Epoch 4/10
3/3 [==============================] - 1s 232ms/step - loss: 2.2767 - acc: 0.3333
Epoch 5/10
3/3 [==============================] - 1s 263ms/step - loss: 2.2717 - acc: 0.0833
Epoch 6/10
3/3 [==============================] - 1s 299ms/step - loss: 2.2556 - acc: 0.0833
Epoch 7/10
3/3 [==============================] - 1s 235ms/step - loss: 2.1795 - acc: 0.3333
Epoch 8/10
3/3 [==============================] - 1s 255ms/step - loss: 2.1299 - acc: 0.4167
Epoch 9/10
3/3 [==============================] - 1s 251ms/step - loss: 2.1008 - acc: 0.3333
Epoch 10/10
3/3 [==============================] - 1s 239ms/step - loss: 2.1128 - acc: 0.1667ETA: 0s - loss: 2.0808 - acc: 0.1250   
